# Transition-based arc-eager unlabeled dependency parser for Ukrainian

## Read the data

Useful links:
* [UD corpus for Ukrainian](https://github.com/UniversalDependencies/UD_Ukrainian-IU/)
* [Easy-to-use library for parsing UD](https://github.com/EmilStenstrom/conllu)

In [3]:
!curl https://raw.githubusercontent.com/UniversalDependencies/UD_Ukrainian-IU/master/uk_iu-ud-train.conllu --output uk_iu-ud-train.conllu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.8M  100 11.8M    0     0  5046k      0  0:00:02  0:00:02 --:--:-- 5046k


In [19]:
!head -n 10 uk_iu-ud-train.conllu

# doc_title = «Я обізвуся до них…»
# newdoc id = 0000
# newpar id = 0001
# sent_id = 0002
# text = У домі римського патриція Руфіна була прегарна фреска, зображення Венери та Адоніса.
# translit = U domi rymśkoho patrycija Rufina bula preharna freska, zobraženńа Venery ta Adonisa.
1	У	у	ADP	Spsl	Case=Loc	2	case	2:case	Id=0003|LTranslit=u|Translit=U
2	домі	дім	NOUN	Ncmsln	Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing	6	obl	6:obl	Id=0004|LTranslit=dim|Translit=domi
3	римського	римський	ADJ	Ao-msgf	Case=Gen|Gender=Masc|Number=Sing	4	amod	4:amod	Id=0005|LTranslit=rymśkyj|Translit=rymśkoho
4	патриція	патрицій	NOUN	Ncmsgy	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	2	nmod	2:nmod	Id=0006|LTranslit=patrycij|Translit=patrycija


In [20]:
!curl https://raw.githubusercontent.com/UniversalDependencies/UD_Ukrainian-IU/master/uk_iu-ud-test.conllu --output uk_iu-ud-test.conllu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2308k  100 2308k    0     0  1643k      0  0:00:01  0:00:01 --:--:-- 1643k


In [21]:
!head -n 10 uk_iu-ud-test.conllu

# doc_title = «Я обізвуся до них…»
# newdoc id = 0000
# newpar id = 01rc
# sent_id = 01rd
# text = Зречення культурної ідентичності – це втрата свободи й самовладності.
# translit = Zrečenńа kuľturnoji identyčnosti – ce vtrata svobody j samovladnosti.
1	Зречення	зречення	NOUN	Ncnsnn	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	6	nsubj	6:nsubj	Id=01re|LTranslit=zrečenńа|Translit=Zrečenńа
2	культурної	культурний	ADJ	Afpfsgf	Case=Gen|Degree=Pos|Gender=Fem|Number=Sing	3	amod	3:amod	Id=01rf|LTranslit=kuľturnyj|Translit=kuľturnoji
3	ідентичності	ідентичність	NOUN	Ncfsgn	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	1	nmod	1:nmod	Id=01rg|LTranslit=identyčnisť|Translit=identyčnosti
4	–	–	PUNCT	U	PunctType=Dash	6	punct	6:punct	Id=01rh|LTranslit=–|Translit=–


In [1]:
from collections import OrderedDict
from conllu import parse
from enum import Enum

with open("uk_iu-ud-train.conllu", "r") as f:
    data = f.read()

trees = parse(data)

In [33]:
tree = trees[0]
print(tree[1])

OrderedDict([('id', 2), ('form', 'домі'), ('lemma', 'дім'), ('upostag', 'NOUN'), ('xpostag', 'Ncmsln'), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Loc'), ('Gender', 'Masc'), ('Number', 'Sing')])), ('head', 6), ('deprel', 'obl'), ('deps', [('obl', 6)]), ('misc', OrderedDict([('Id', '0004'), ('LTranslit', 'dim'), ('Translit', 'domi')]))])


In [3]:
for node in tree:
    head = node["head"]
    print("{} <-- {}".format(node["form"],
                             tree[head - 1]["form"]
                             if head > 0 else "root"))

У <-- домі
домі <-- була
римського <-- патриція
патриція <-- домі
Руфіна <-- патриція
була <-- root
прегарна <-- фреска
фреска <-- була
, <-- зображення
зображення <-- фреска
Венери <-- зображення
та <-- Адоніса
Адоніса <-- Венери
. <-- була


## Design actions and the oracle

We will be using a static oracle that reproduces a single valid order of actions.

In [196]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"
    SWAP = "swap"

In [204]:
def intersects(n1, n2):
    if n1['head'] > n2['id'] and n2['head'] > n1['head']:
        return True
                
    if n1['head'] > n2['id'] and n2['head'] < n1['id']:
        return True            

    if n2['head'] < n1['id'] and n1['head'] < n2['head']:
        return True

    if n2['head'] > n1['id'] and n1['head'] < n2['id'] and n2['head'] < n1['head']:
        return True
    
    return False

def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    elif top_stack and top_queue and top_stack['id'] != 0 and intersects(top_stack, top_queue):        
        return Actions.SWAP
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [205]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

def trace_actions(tree, log=True):
    """
    Try out the oracle to verify it's returning the right actions.
    """
    stack, queue, relations = [ROOT], tree[:], []
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        if log:
            print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
            print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
            print("Relations:", relations)
            print(action)
            print("========================")
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        elif action == Actions.SWAP:
            stack.append(queue.pop(0))
            tmp = stack[-1]
            stack[-1] = stack[-2]
            stack[-2] = tmp
        else:
            print("Unknown action.")
    if log:
        print("Gold relations:")
        print([(node["id"], node["head"]) for node in tree])
        print("Retrieved relations:")
        print(sorted(relations))

trace_actions(tree)

Stack: ['ROOT_0']
Queue: ['Після_1', 'першого_2', 'репортажу_3', ',_4', 'готуючись_5', 'до_6', 'наступних_7', ',_8', 'раптом_9', 'від_10', 'колег_11', 'дізнаюся_12', ',_13', 'що_14', '«_15', 'зверху_16', '»_17', '—_18', 'з_19', 'НТКУ_20', ',_21', 'від_22', 'пана_23', 'Аласанії_24', ',_25', 'прислано_26', '«_27', 'їхнього_28', '»_29', 'коментатора_30', ',_31', 'а_32', 'мій_33', 'договір_34', 'анулюють_35', '»_36', ',_37', '—_38', 'написав_39', 'Довгич_40', '._41']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'Після_1']
Queue: ['першого_2', 'репортажу_3', ',_4', 'готуючись_5', 'до_6', 'наступних_7', ',_8', 'раптом_9', 'від_10', 'колег_11', 'дізнаюся_12', ',_13', 'що_14', '«_15', 'зверху_16', '»_17', '—_18', 'з_19', 'НТКУ_20', ',_21', 'від_22', 'пана_23', 'Аласанії_24', ',_25', 'прислано_26', '«_27', 'їхнього_28', '»_29', 'коментатора_30', ',_31', 'а_32', 'мій_33', 'договір_34', 'анулюють_35', '»_36', ',_37', '—_38', 'написав_39', 'Довгич_40', '._41']
Relations: []
Actions.SHIFT
Stack: [

## Feature extraction

Reference: [Dependency Parsing by Kübler, McDonald, and Nivre](https://books.google.com.ua/books?id=k3iiup7HB9UC&pg=PA21&hl=uk&source=gbs_toc_r&cad=4#v=onepage&q&f=false)

In [225]:
def extract_features(stack, queue, relations):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
        features["s0-rdep"] = len([r for r in relations if r[1] == stack_top['id']]) > 0
        features["s0-ldep"] = len([r for r in relations if r[0] == stack_top['id']]) > 0
        #features["deprel"] = stack_top["deprel"] if stack_top["deprel"] else "NONE_DEP"
        if stack_top["feats"]:
            for k, v in stack_top["feats"].items():
                features["s0-" + k] = v
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        if queue_top["feats"]:
            for k, v in queue_top["feats"].items():
                features["q0-" + k] = v
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    if stack and queue:
        features["distance"] = queue[0]["id"] - stack[-1]["id"]
    return features

## Prepare train and test data

In [226]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        features.append(extract_features(stack, queue, relations))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        elif action == Actions.SWAP:
            stack.append(queue.pop(0))
            tmp = stack[-1]
            stack[-1] = stack[-2]
            stack[-2] = tmp    
        else:
            print("Unknown action.")
    return features, labels

In [224]:
features, labels = get_data(tree)
print("Number of words:", len(tree))
print("Number of actions:", len(labels))
print("List of actions taken:", labels)
print("Features:")
for word in features:
    print(word)

Number of words: 20
Number of actions: 41
List of actions taken: ['shift', 'left', 'shift', 'left', 'right', 'shift', 'left', 'right', 'right', 'right', 'shift', 'shift', 'left', 'left', 'reduce', 'right', 'shift', 'shift', 'shift', 'left', 'left', 'shift', 'left', 'left', 'reduce', 'right', 'right', 'right', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 's0-rdep': False, 's0-ldep': False, 'q0-word': 'У', 'q0-lemma': 'у', 'q0-tag': 'ADP', 'q0-Case': 'Gen', 'q1-word': 'нас', 'q1-tag': 'PRON', 'q2-tag': 'VERB', 'q3-tag': 'ADJ', 'distance': 1}
{'s0-word': 'У', 's0-lemma': 'у', 's0-tag': 'ADP', 's0-rdep': False, 's0-ldep': False, 's0-Case': 'Gen', 's1-tag': 'ROOT', 'q0-word': 'нас', 'q0-lemma': 'ми', 'q0-tag': 'PRON', 'q0-Animacy': 'Anim', 'q0-Case': 'Gen', 'q0-Number': 'Plur', 'q0-Person': '1', 'q0-PronType': 'Prs', 'q1-word': 'є', 'q1-tag': 

In [86]:
def is_broken(tree):
    for n in tree:
        if type(n["id"]) != int:
            return True
    return False    

In [87]:
broken = list(filter(is_broken, trees))

In [88]:
len(broken)

177

In [105]:
for i, n in enumerate(broken[2]):
    print(i, n['id'], n['head'], n['form'])

0 1 2 Мєдвєдєв
1 2 0 співав
2 3 4 «
3 4 2 Гетьмани
4 5 4 »
5 6 8 ,
6 7 8 «
7 8 4 Олесю
8 9 8 »
9 10 12 ,
10 11 12 «
11 12 4 Дощик
12 13 12 »
13 14 16 ,
14 15 16 хор
15 16 2 співав
16 17 20 «
17 18 20 Гей
18 19 20 не
19 20 16 дивуйте
20 21 20 »
21 22 23 ,
22 23 2 співачка
23 24 23 Снарская
24 25 27 (
25 26 27 наша
26 27 24 волинячка
27 28 27 )
28 (28, '.', 1) None співала
29 29 34 –
30 30 34 «
31 31 34 Чом
32 32 33 ,
33 33 31 чом
34 34 23 чорнобров
35 35 34 »
36 36 42 ,
37 37 42 а
38 38 40 в
39 39 40 самому
40 40 42 кінці
41 41 42 хор
42 42 2 утяв
43 43 44 «
44 44 42 Српиjа
45 45 44 свободна
46 46 44 »
47 47 51 (
48 48 51 «
49 49 51 Ще
50 50 51 не
51 51 44 вмерла
52 52 51 Україна
53 53 51 »
54 54 51 )
55 55 2 .


In [103]:
broken[2]

TokenList<Мєдвєдєв, співав, «, Гетьмани, », ,, «, Олесю, », ,, «, Дощик, », ,, хор, співав, «, Гей, не, дивуйте, », ,, співачка, Снарская, (, наша, волинячка, ), співала, –, «, Чом, ,, чом, чорнобров, », ,, а, в, самому, кінці, хор, утяв, «, Српиjа, свободна, », (, «, Ще, не, вмерла, Україна, », ), .>

In [104]:
broken[2][28]

OrderedDict([('id', (28, '.', 1)),
             ('form', 'співала'),
             ('lemma', 'співати'),
             ('upostag', 'VERB'),
             ('xpostag', 'Vmpis-sf'),
             ('feats',
              OrderedDict([('Aspect', 'Imp'),
                           ('Gender', 'Fem'),
                           ('Mood', 'Ind'),
                           ('Number', 'Sing'),
                           ('Tense', 'Past'),
                           ('VerbForm', 'Fin')])),
             ('head', None),
             ('deprel', '_'),
             ('deps', [('root', 0), ('conj', 2)]),
             ('misc',
              OrderedDict([('Id', '3cbh'),
                           ('LTranslit', 'spivaty'),
                           ('Translit', 'spivala')]))])

In [90]:
trees[0]

TokenList<У, домі, римського, патриція, Руфіна, була, прегарна, фреска, ,, зображення, Венери, та, Адоніса, .>

In [163]:
def non_projective(tree):
    for n1 in tree:
        for n2 in tree:
            if n1['id'] < n2['id'] and intersects(n1, n2):
                return True
            
    return False

In [82]:
len(trees)

5290

In [110]:
clean_trees = list(filter(lambda t: not is_broken(t), trees))

In [137]:
len(clean_trees)

5113

In [164]:
projective_trees = list(filter(lambda t: not non_projective(t), clean_trees))

In [165]:
len(projective_trees)

4719

In [185]:
trace_actions(projective_trees[0])

Stack: ['ROOT_0']
Queue: ['У_1', 'домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.SHIFT
OrderedDict([('id', 1), ('form', 'У'), ('lemma', 'у'), ('upostag', 'ADP'), ('xpostag', 'Spsl'), ('feats', OrderedDict([('Case', 'Loc')])), ('head', 2), ('deprel', 'case'), ('deps', [('case', 2)]), ('misc', OrderedDict([('Id', '0003'), ('LTranslit', 'u'), ('Translit', 'U')]))]) OrderedDict([('id', 2), ('form', 'домі'), ('lemma', 'дім'), ('upostag', 'NOUN'), ('xpostag', 'Ncmsln'), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Loc'), ('Gender', 'Masc'), ('Number', 'Sing')])), ('head', 6), ('deprel', 'obl'), ('deps', [('obl', 6)]), ('misc', OrderedDict([('Id', '0004'), ('LTranslit', 'dim'), ('Translit', 'domi')]))])
Stack: ['ROOT_0', 'У_1']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11',

In [227]:
train_features, train_labels = [], []
for tree in clean_trees:
    tree_features, tree_labels = get_data(tree)
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

173547 173547


In [228]:
# Test data

with open("uk_iu-ud-test.conllu", "r") as f:
    data = f.read()
test_trees = list(filter(lambda t: not is_broken(t), parse(data)))

test_features, test_labels = [], []
for tree in test_trees:
    tree_features, tree_labels = get_data(tree)
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

32394 32394


## Train a classifier

In [229]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [230]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

print("\nTotal number of features: ", len(vec.get_feature_names()))


Total number of features:  105019


In [231]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

#print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

In [232]:
lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial", n_jobs = 6, max_iter=3000, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


convergence after 1481 epochs took 387 seconds


[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:  6.4min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=3000, multi_class='multinomial',
          n_jobs=6, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=1, warm_start=False)

In [233]:
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

        left       0.93      0.96      0.95      7969
      reduce       0.89      0.85      0.87      8651
       right       0.81      0.83      0.82      7669
       shift       0.89      0.90      0.89      8044
        swap       0.75      0.10      0.17        61

   micro avg       0.88      0.88      0.88     32394
   macro avg       0.86      0.73      0.74     32394
weighted avg       0.88      0.88      0.88     32394



## Calculate the unlabeled attachment score
UAS - the percentage of words in an input that are assigned the correct head.

In [236]:
def dep_parse(sentence, oracle, vectorizer, log=True):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features(stack, queue, relations)
            action = oracle.predict(vectorizer.transform([features]))[0]
            if log:
                print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
                print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
                print("Relations:", relations)
                print(action)
                print("========================")
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))            
            elif action == Actions.SWAP:
                stack.append(queue.pop(0))
                tmp = stack[-1]
                stack[-1] = stack[-2]
                stack[-2] = tmp  
            else:
                print("Unknown action.")
    return sorted(relations)

In [ ]:
total, tp = 0, 0
for tree in test_trees:    
    golden = [(node["id"], node["head"]) for node in tree]
    predicted = dep_parse(tree, lrc, vec, False)
    total += len(tree)
    tp += len(set(golden).intersection(set(predicted)))

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))